# Check sparse implementation

In [1]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

## get_infection_probas in inference_model

In [2]:
N = 500
i = [0, 1]
j = [10, 2]
rates = [0.5, 0.8]
transmissions_list = list(zip(i, j, rates))
transmissions_csr = coo_matrix((rates, (i, j)), shape=(N, N)).tocsr()
probas = np.ones((N, 3))/3.

In [3]:
# coo then csr is faster
%timeit transmissions_csr = coo_matrix((rates, (i, j)), shape=(N, N)).tocsr()

133 µs ± 7.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%timeit transmissions_csr = csr_matrix((rates, (i, j)), shape=(N, N))

207 µs ± 5.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
def get_infection_probas_list(probas, transmissions):
    """
    - probas[i,s] = P_s^i(t)
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = sum_j lambda_ij P_I^j(t)
    """
    N = probas.shape[0]
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            probas[j, 1]*rate
            for i0, j, rate in transmissions if i0 == i
        ])
        infection_probas[i] = rates.sum()
    return infection_probas

In [6]:
def get_infection_probas_fast(probas, transmissions):
    """
    - probas[i,s] = P_s^i(t)
    - transmissions = csr sparse matrix of i, j, lambda_ij
    - infection_probas[i]  = sum_j lambda_ij P_I^j(t)
    """
    N = probas.shape[0]
    infection_probas = transmissions.dot(probas[:, 1])
    return infection_probas

In [7]:
p_old = get_infection_probas_list(probas, transmissions_list)
p_new = get_infection_probas_fast(probas, transmissions_csr)
np.all(p_old==p_new)

True

In [8]:
%timeit p_old = get_infection_probas_list(probas, transmissions_list)

2 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit p_new = get_infection_probas_fast(probas, transmissions_csr)

7.28 µs ± 42.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## get_infection_probas in sir_model

In [10]:
N = 500
i = [0, 1, 3, 3]
j = [10, 2, 12, 13]
rates = [0.5, 0.8, 0.9, 0.6]
transmissions_list = list(zip(i, j, rates))
transmissions_csr = coo_matrix((rates, (i, j)), shape=(N, N)).tocsr()
states = np.ones(N)

In [11]:
def get_infection_probas_list(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    """
    infected = (states == 1)
    N = len(states)
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            rate for i0, j, rate in transmissions
            if i0 == i and infected[j]
        ])
        infection_probas[i] = 1 - np.prod(1 - rates)
    return infection_probas

In [12]:
def get_infection_probas_fast(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = csr sparse matrix of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    We use prod_j  [1 - lambda_ij] = exp(sum_j log(1 - lambda_ij)) 
    """
    infected = (states == 1)
    infected_transmissions = transmissions.multiply(infected)
    infection_probas = 1 - np.exp(
        infected_transmissions.multiply(-1).log1p().sum(axis=1)
    )
    infection_probas = np.array(infection_probas).squeeze()
    return infection_probas

In [13]:
p_old = get_infection_probas_list(states, transmissions_list)
p_new = get_infection_probas_fast(states, transmissions_csr)
np.all(p_old==p_new)

True

In [14]:
for x in set(i):
    print(f"i={x} : {p_new[x]}")

i=0 : 0.5
i=1 : 0.8
i=3 : 0.96


In [15]:
%timeit p_old = get_infection_probas_list(states, transmissions_list)

3.81 ms ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit p_new = get_infection_probas_fast(states, transmissions_csr)

450 µs ± 53.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
